# Part 5: More realistic trajectories [optional]

Import library

In [ ]:
from vpython import vector, sphere, cylinder, curve, color, graph, gcurve, gvbars, gdots, canvas, rate, dot, mag

# Create objects for display

Let's set the scene, create Earth and Moon as spheres, the ship as a cylinder and a trail that follows the ship.

In [ ]:
moonlocation=vector(4.0e8,0,0)


In [ ]:
scene=canvas(title="Voyage to the Moon",
              width=800,height=600,
              center=moonlocation/1.8)

In [ ]:
earth = sphere(pos=vector(0,0,0), radius=6.4e6, color=color.blue)
moon  = sphere(pos=moonlocation,radius=1.76e3, color=color.cyan)
ship  = cylinder(axis=vector(5e6,0,0), radius=2e6)
trail = curve(color=ship.color)

# Create Graphs for the energy and work display
We will assign them the time-dependent values of energies and work later inside the loop.

In [ ]:
energyplot = graph(x=0,y=200,xmin=0,xmax=moonlocation.x,
                      ymin=-2E10,ymax=2E10,
                      xtitle='Ship Position',
                      ytitle='Energy')
U_Graph=gcurve(color=color.blue)
K_Graph=gcurve(color=color.green)
W_Graph=gdots(color=color.black, size=0.1)
KU_Graph=gvbars(color=color.red)

In [ ]:
G=6.67E-11
initial_speed = 10460. 
earth.mass = 6.0e24
ship.mass = 173
dt = 100
moon.mass = 7.0e22
moon.radius = 1.76e6
t = 0
Work = 0.
crash = 0
scene.autoscale=0

A more realistic trajectory would be to shoot the moon tangential to the surface of the Earth from its other side. We need to change the ship's momentum and position. Find the differences below:

In [ ]:
ship.pos = vector(-(earth.radius+50000),0,0)         # Change the ship's position to the other side of Earth.
ship.Vmomentum = vector(0,ship.mass*initial_speed,0) # Change the momentum → the initial velocity is now in O → +y

In [ ]:
while (crash==0):
    rate(100)
    Vr1=ship.pos-earth.pos
    VForceEarth = -G*earth.mass*ship.mass*ship.pos/mag(ship.pos)**3 
    Vr2=ship.pos-moon.pos
    VForceMoon  = -G*moon.mass*ship.mass*Vr2/mag(Vr2)**3
    ship.Vforce    = VForceEarth + VForceMoon
    Vdelta_r = (ship.Vmomentum/ship.mass)*dt
    ship.Vmomentum = ship.Vmomentum + ship.Vforce*dt
    ship.pos      = ship.pos + Vdelta_r
    UEarthShip= -G*earth.mass*ship.mass/mag(ship.pos) 
    UMoonShip = -G*moon.mass*ship.mass/mag(Vr2)
    U=UEarthShip+UMoonShip
    K = mag(ship.Vmomentum)**2/(2.*ship.mass)
    Work = Work + dot(ship.Vforce,Vdelta_r)
    U_Graph.plot(pos=(ship.pos.x,U))
    K_Graph.plot(pos=(ship.pos.x,K))
    KU_Graph.plot(pos=(ship.pos.x,K+U))
    W_Graph.plot(pos=(ship.pos.x,Work))

    trail.append(pos=ship.pos)
    t=t+dt
    if (mag(ship.pos-earth.pos) <= earth.radius ):
        print ("ship crashed back on the earth")
        crash=1
    elif (mag(ship.pos -moon.pos )< (moon.radius+50000) ) :
        print ("ship crashed on the moon at time ",t, "seconds")
        print ("ship's initial speed ", initial_speed)
        print ("ship's final speed ", mag(ship.Vmomentum)/ship.mass)
        crash=1